### Facial Recognition
This code was run in Colab.

In [ ]:
!pip install face_recognition

In [ ]:
# import the necessary packages
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
from google.colab import drive

Training the Model

In [ ]:
args = {
    "dataset": "dataset",
    "detection_method": "cnn",
    "encodings": "encodings.pickle"
}

In [ ]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(args["dataset"]))
# initialize the list of known encodings and known names
knownEncodings = []
knownNames = []

print(imagePaths)

[INFO] quantifying faces...
['dataset/kejriwal/kejriwal_3.jpg', 'dataset/kejriwal/kejriwal_5.jpg', 'dataset/kejriwal/kejriwal_4.jpg', 'dataset/kejriwal/kejriwal_6.jpg', 'dataset/kejriwal/kejriwal_2.jpg', 'dataset/kejriwal/kejriwal_1.jpg', 'dataset/yadav/yadav_1.jpg', 'dataset/yadav/yadav_5.jpg', 'dataset/yadav/yadav_2.jpg', 'dataset/yadav/yadav_3.jpg', 'dataset/yadav/yadav_4.jpg', 'dataset/owaisi/owaisi_5.jpg', 'dataset/owaisi/owaisi_4.jpg', 'dataset/owaisi/owaisi_1.jpg', 'dataset/owaisi/owaisi_3.jpg', 'dataset/owaisi/owaisi_2.jpg', 'dataset/modi/modi_5.jpg', 'dataset/modi/modi_3.jpg', 'dataset/modi/modi_2.jpg', 'dataset/modi/modi_1.jpg', 'dataset/modi/modi_4.jpg', 'dataset/modi/modi_6.jpg', 'dataset/gandhi/gandhi_3.jpg', 'dataset/gandhi/gandhi_1.jpg', 'dataset/gandhi/gandhi_4.jpg', 'dataset/gandhi/gandhi_5.jpg', 'dataset/gandhi/gandhi_6.jpg', 'dataset/gandhi/gandhi_2.jpg']


In [ ]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]
	# load the input image and convert it from BGR (OpenCV ordering)
	# to dlib ordering (RGB)
	image = cv2.imread(imagePath)
	rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

	# detect the (x, y)-coordinates of the bounding boxes
	# corresponding to each face in the input image
	boxes = face_recognition.face_locations(rgb,
		model=args["detection_method"])
	# compute the facial embedding for the face
	encodings = face_recognition.face_encodings(rgb, boxes)
	# loop over the encodings
	for encoding in encodings:
		# add each encoding + name to our set of known names and
		# encodings
		knownEncodings.append(encoding)
		knownNames.append(name)

# dump the facial encodings + names to disk
print("[INFO] serializing encodings...")
data = {"encodings": knownEncodings, "names": knownNames}
f = open(args["encodings"], "wb")
f.write(pickle.dumps(data))
f.close()

Using the Model

In [ ]:
# import the necessary packages
from imutils.video import FileVideoStream
import face_recognition
import imutils
import pickle
import time
import cv2
from google.colab.patches import cv2_imshow
import os
import pandas as pd
import json
from datetime import datetime

In [ ]:
basepath = "drive/MyDrive/Capstone/Videos/videos_1"

In [ ]:
file_paths = pd.read_csv("drive/MyDrive/Capstone/fr_sample.csv").filepath.str.replace("video_", "videos_")

In [ ]:
video_data = pd.DataFrame(columns=["filename", "names", "counts"])

args = {
	"encodings": "encodings.pickle",
	"display": 0,
	"detection_method": "cnn",
	"output": None,
}

# load the known faces and embeddings
print("[INFO] loading encodings...")
data = pickle.loads(open(args["encodings"], "rb").read())

# for file_num, filename in enumerate(ordered_files[85:100]):
for file_num, filename in enumerate(file_paths[85:]):
	if filename == ".ipynb_checkpoints":
		continue
	starttime = datetime.now()

	# args["input"] = f"{basepath}/{filename}"
	args["input"] = filename

	# initialize the video stream and pointer to output video file, then
	# allow the camera sensor to warm up

	print(f"[INFO] starting video stream for {filename}...")

	vs = FileVideoStream(args["input"]).start()

	writer = None
	time.sleep(2.0)
	num_frames_looped=0

	# loop over frames from the video file stream
	while True:
		# grab the frame from the threaded video stream
		frame = vs.read()
		if frame is None:
			break

		while num_frames_looped % 10 != 0:
			frame = vs.read()
			if frame is None:
				break
			num_frames_looped+=1
		num_frames_looped+=1
		print(f"on frame {num_frames_looped}")
		# convert the input frame from BGR to RGB then resize it to have
		# a width of 750px (to speedup processing)
		try:
			rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
		except:
			print(f"breaking after {num_frames_looped}")
			break
		rgb = imutils.resize(frame, width=750)
		r = frame.shape[1] / float(rgb.shape[1])
		# detect the (x, y)-coordinates of the bounding boxes
		# corresponding to each face in the input frame, then compute
		# the facial embeddings for each face
		boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
		encodings = face_recognition.face_encodings(rgb, boxes)
		names = []

			# loop over the facial embeddings
		for encoding in encodings:
			# attempt to match each face in the input image to our known
			# encodings
			matches = face_recognition.compare_faces(data["encodings"],
				encoding)
			name = "Unknown"
			# check to see if we have found a match
			if True in matches:
				# find the indexes of all matched faces then initialize a
				# dictionary to count the total number of times each face
				# was matched
				matchedIdxs = [i for (i, b) in enumerate(matches) if b]
				counts = {}
				# loop over the matched indexes and maintain a count for
				# each recognized face face
				for i in matchedIdxs:
					name = data["names"][i]
					counts[name] = counts.get(name, 0) + 1
				# determine the recognized face with the largest number
				# of votes (note: in the event of an unlikely tie Python
				# will select first entry in the dictionary)
				name = max(counts, key=counts.get)

			# update the list of names
			names.append(name)
				# loop over the recognized faces

		try:
			video_data.loc[file_num] = [filename[:-4], str(names), json.dumps(counts)]
		except:
			video_data.loc[file_num] = [filename[:-4], str(names), '{}']
		video_data.to_csv(f"drive/MyDrive/Capstone/fr_output/fr_group_2_85_100.csv")
	# do a bit of cleanup
	cv2.destroyAllWindows()
	vs.stop()
	print(datetime.now() - starttime)

video_data.to_csv(f"drive/MyDrive/Capstone/fr_output/fr_group_2__85_100.csv")
video_data